In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import rioxarray 
import os

In [2]:
crops = ['Corn','Soybean', "Wheat", "Rice"]
phases = [2]

In [3]:
# Read in the data
gdf_dict = {}
for crop in crops:
    for phase in phases:
        rds = rioxarray.open_rasterio(
            "../" + os.sep + "processed" + os.sep + "output" + os.sep + crop + "_phase_" + str(phase) + "_rc_mean.asc"
        )
        rds = rds.squeeze().drop("spatial_ref").drop("band")
        rds.name = "data"
        df = rds.to_dataframe().reset_index().replace(-9, np.nan).dropna()
        # Convert to geodataframe
        gdf_dict[crop+str(phase)] = df

In [4]:
def dataframe_to_geopandas(df):
    # Convert the x and y columns to longitude and latitude, respectively
    geometry = [Point(x, y) for x, y in zip(df['x'], df['y'])]
    
    # Create a GeoDataFrame from the geometry and data columns
    gdf = gpd.GeoDataFrame(df['data'], geometry=geometry)

    # Convert to winkel tripel projection
    gdf.crs = "EPSG:4326"
    gdf = gdf.to_crs("+proj=wintri")

    return gdf

In [5]:
# Convert the dataframes to geodataframes
gdf_dict_gpd = {}
for i in gdf_dict.keys():
    gdf_dict_gpd[i] = dataframe_to_geopandas(gdf_dict[i])


In [6]:
# Read in the country borders
countries = gpd.read_file("../" + os.sep + "spatial_data" + os. sep +"countries" + os.sep + "World_Countries__Generalized_.shp")
countries.crs = "EPSG:4326"
countries = countries.to_crs('+proj=wintri')

In [7]:
# Iterate over the geodataframes and clip them to the country borders 
# And calculate the mean value of the data within the country borders
# Save mean values for the countries to a new dataframe and save it to a csv file
for crop in crops:
    for phase in phases:
        crop_gdf = gdf_dict_gpd[crop+str(phase)]
        countries_with_crops = gpd.sjoin(countries, crop_gdf, how="inner", op='intersects')
        write_header = True
        for country in set(countries_with_crops['COUNTRY']):
            country_gdf = countries_with_crops[countries_with_crops['COUNTRY'] == country]
            mean_value = country_gdf['data'].mean()
            print(country, mean_value)
            with open("../" + os.sep + "processed" + os.sep + "output" + os.sep + "country_means" + os.sep + crop + str(phase) + "mean_values.csv", "a") as f:
                if write_header:
                    f.write("country,mean_value\n")
                    write_header = False
                f.write(country + "," + str(mean_value*100) + "\n")



/home/florian/local/anaconda3/envs/seaweed-growth-model/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Poland -0.42925864
Senegal -0.11645072
Moldova -0.090738736
Bhutan -0.306748
Chad -0.011620763
Guinea-Bissau 0.0
Australia -0.39839414
Serbia -0.3660705
Azerbaijan -0.21118718
Madagascar -0.040954825
Netherlands -0.6116485
Georgia -0.001653489
Iraq -0.08636007
Austria -0.5854733
Cuba -0.01653936
United States -0.51205474
Morocco -0.014873774
Ukraine -0.32501328
South Korea -0.40531415
Chile -0.6073579
Czech Republic -0.44120318
Togo -0.013442351
Cameroon -0.1473582
Mexico -0.2211501
Côte d'Ivoire -0.0535618
Mauritania 0.0
Central African Republic 0.0
Thailand -0.16381656
North Macedonia -0.21137923
Hungary -0.39291513
Benin -0.010344538
Montenegro -0.44515738
Russian Federation -0.13221194
Turkmenistan 0.0
Honduras -0.056039218
Egypt -0.66834205
Syria -0.13161534
Greece -0.5355459
Somalia -0.054801896
Malawi -0.24991292
Uruguay -0.06397448
Bolivia -0.25670922
Nicaragua -0.010549765
Slovenia -0.4934559
Bangladesh -0.13481294
El Salvador -0.38654894
Venezuela -0.25745553
Sri Lanka -0.087

/home/florian/local/anaconda3/envs/seaweed-growth-model/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Japan -0.043614767
Paraguay -0.3449566
Uruguay -0.23938806
Brazil -0.4273134
Bolivia -0.04438176
Slovenia -0.4757658
Turkey -0.23946588
Moldova -0.1101239
Laos -0.0062974147
Argentina -0.31716663
Serbia -0.43032175
Nigeria -0.0013932579
Canada -0.3844419
Romania -0.27016646
Ecuador -0.09682867
Austria -0.46825817
Iran -0.3956632
Albania -0.12906206
United States -0.381662
North Korea -0.024885168
Bosnia and Herzegovina -0.31742883
Ukraine -0.24332032
Nepal -0.1232446
Uganda -0.003486683
South Korea -0.09982126
Indonesia -0.035777677
Myanmar -0.08635248
Mexico -0.026399884
Zimbabwe -0.06335761
Thailand -0.15652375
China -0.16270654
Rwanda -0.009587429
Italy -0.47046778
Cambodia -0.029056469
Colombia -0.14637558
Hungary -0.4556692
Tanzania 0.0
Benin 0.0
India -0.1235055
Slovakia -0.2009895
Russian Federation -0.18152227
France -0.4510687
Croatia -0.37113985
Zambia -0.26496938
South Africa -0.1913079
Vietnam -0.1018648


/home/florian/local/anaconda3/envs/seaweed-growth-model/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Japan -0.16577032
Poland -0.51244724
Armenia -0.31690934
Jordan 0.0
Moldova -0.32365826
Bhutan -0.4229138
Australia -0.34400153
Serbia -0.49508086
Azerbaijan -0.22153683
Netherlands -0.71882415
Georgia -0.000825341
Iraq -0.19408761
Austria -0.60726374
United States -0.41432226
Morocco -0.23370865
Ukraine -0.4261557
Chile -0.63719183
Czech Republic -0.5529134
Mexico -0.58001167
Denmark -0.6643078
North Macedonia -0.39621186
Hungary -0.5224952
Russian Federation -0.17099135
Turkmenistan -0.18316974
Ireland -0.74142456
Egypt -0.6736691
Syria -0.32725635
Greece -0.2958771
Norway -0.4095214
Uruguay -0.46621126
Bolivia -0.07564715
Slovenia -0.47942072
Bangladesh -0.094387114
Romania -0.38894466
Ecuador -0.017808009
Yemen -0.3445259
Israel -0.19770995
New Zealand -0.61327094
Kenya -0.25715074
Latvia -0.35453343
Estonia -0.26945812
Nepal -0.24644072
Libya -0.00030406142
Myanmar -0.11044377
Rwanda -0.055375393
Italy -0.33184737
Kazakhstan -0.034888174
Kyrgyzstan -0.14383116
Colombia -0.2149043


/home/florian/local/anaconda3/envs/seaweed-growth-model/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Japan -0.1943894
Senegal -0.30406475
Bhutan -0.10412957
Chad 0.0
Guinea-Bissau -0.10776578
Australia -0.5543886
Madagascar -0.1447075
Iraq -0.3317062
Cuba -0.06713452
Guyana -0.18568188
United States -0.47989163
Morocco -0.08597795
South Korea -0.22009504
Chile -0.17227527
Togo -0.19530454
Cameroon -0.034783922
Mexico -0.50475633
Côte d'Ivoire -0.045495566
Mauritania -0.36885607
Thailand -0.1681402
North Macedonia -0.40056062
Liberia 0.0
Benin -0.1922033
Russian Federation -0.030248903
Turkmenistan 0.0
Honduras -0.51276433
Egypt -0.6691532
Greece -0.31963572
Malawi -0.038393445
Uruguay -0.45826715
Bolivia -0.020117585
Nicaragua -0.43958065
Bangladesh -0.35434532
El Salvador -0.66608745
Venezuela -0.27501574
Sri Lanka -0.19262637
Ecuador -0.28372002
Kenya -0.1800231
Dominican Republic -0.21278468
Nepal -0.11687231
Timor-Leste -0.20296544
Myanmar -0.2534198
Guatemala -0.28191698
Rwanda -0.58898014
Italy -0.3540604
Kyrgyzstan -0.11090794
Colombia -0.15817761
Pakistan -0.13373552
Mozambiqu

In [8]:
# Do a spatial join between the country borders and the crop data
countries_with_crops = gpd.sjoin(countries, crop_gdf, how="inner", op='intersects')
countries_with_crops.head()

/home/florian/local/anaconda3/envs/seaweed-growth-model/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,SHAPE_Leng,SHAPE_Area,geometry,index_right,data
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3958221,-0.742757
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3958203,-0.526730
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3936608,-0.662635
10,11,El Salvador,SV,El Salvador,SV,7.253572,1.719035,"POLYGON ((-9659155.752 1617456.549, -9663767.8...",3932288,-0.732228
11,12,Guatemala,GT,Guatemala,GT,16.760150,9.193015,"POLYGON ((-9831388.759 1692437.600, -9841243.4...",3876108,-0.416150


In [9]:
for country in set(countries_with_crops['COUNTRY']):
    print(country)
    print(countries_with_crops[countries_with_crops['COUNTRY'] == country]['data'].mean())


Japan
-0.1943894
Senegal
-0.30406475
Bhutan
-0.10412957
Chad
0.0
Guinea-Bissau
-0.10776578
Australia
-0.5543886
Madagascar
-0.1447075
Iraq
-0.3317062
Cuba
-0.06713452
Guyana
-0.18568188
United States
-0.47989163
Morocco
-0.08597795
South Korea
-0.22009504
Chile
-0.17227527
Togo
-0.19530454
Cameroon
-0.034783922
Mexico
-0.50475633
Côte d'Ivoire
-0.045495566
Mauritania
-0.36885607
Thailand
-0.1681402
North Macedonia
-0.40056062
Liberia
0.0
Benin
-0.1922033
Russian Federation
-0.030248903
Turkmenistan
0.0
Honduras
-0.51276433
Egypt
-0.6691532
Greece
-0.31963572
Malawi
-0.038393445
Uruguay
-0.45826715
Bolivia
-0.020117585
Nicaragua
-0.43958065
Bangladesh
-0.35434532
El Salvador
-0.66608745
Venezuela
-0.27501574
Sri Lanka
-0.19262637
Ecuador
-0.28372002
Kenya
-0.1800231
Dominican Republic
-0.21278468
Nepal
-0.11687231
Timor-Leste
-0.20296544
Myanmar
-0.2534198
Guatemala
-0.28191698
Rwanda
-0.58898014
Italy
-0.3540604
Kyrgyzstan
-0.11090794
Colombia
-0.15817761
Pakistan
-0.13373552
Mozambiqu